In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import platform
import pathlib  
import os.path
import netCDF4 as nc
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import yaml

from superflexpy.implementation.elements.gr4j import BaseElement, UnitHydrograph1, UnitHydrograph2, FluxAggregator 
from superflexpy.implementation.elements.gr4j import InterceptionFilter, ProductionStore, RoutingStore
from superflexpy.implementation.elements.structure_elements import Transparent, Splitter, Junction
from superflexpy.implementation.root_finders.pegasus import PegasusPython
from superflexpy.implementation.numerical_approximators.implicit_euler import ImplicitEulerPython
from superflexpy.framework.unit import Unit
from superflexpy.framework.node import Node
from superflexpy.framework.network import Network
# from utils.precipitation_evaporation import *

In [2]:
# Import the correct files from computer

cwd = pathlib.Path().resolve()
src = cwd.parent
data = src.parent.parent.parent
OS_type = platform.system()

if OS_type == 'Darwin':
    username = 'Mats '
    data_path = os.path.join(data, 'data_tana', 'catchments')
    shape_path = os.path.join(data, 'data_tana', 'catchments')
    
else:
    username = 'Mootje'
    data_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'Mini_data')
    connection_path = os.path.join(data, 'Desktop', 'MDP_Github', 'Git_mdp', 'data', 'catchments')
    # shape_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'Shapefiles','Mini_shapes')

print(f"Welcome {username}, have a wondeful day on your {OS_type} machine. Your data should be located in {data_path}")
# print(data)

Welcome Mootje, have a wondeful day on your Windows machine. Your data should be located in C:\Users\Moniek van Zon\OneDrive - Delft University of Technology\TU Delft\Master ENVM\MDP\Model\Data\Mini_data


In [3]:
# Read the files and add the file name as name to the dictionary

data_files = glob.glob(os.path.join(data_path, '*.nc'))
datasets = {}
file_identifier_list = []

for file_path in data_files:
    # Extract the file identifier from the file name
    file_name = os.path.basename(file_path)  # Get just the file name

    # Split the file name correctly so you will only have the part you want
    file_identifier = file_name.split('_')[1] 
    file_identifier2 = file_identifier.split('.')[0]
    file_identifier_list.append(file_identifier2)
   
    # Open the dataset (NetCDF)
    dataset = xr.open_dataset(file_path)
    
    # Add the file identifier as a new coordinate
    dataset = dataset.assign_coords(file_identifier=file_identifier)
    
    # Add the dataset to the dictionary with the file identifier as the key
    datasets[file_identifier2] = dataset



In [4]:
def GR4J_element(x1,x2,x3,x4, data_set, file_name):
    # Initial guess for the parameters
    #x1, x2, x3, x4 = (50.0, 0.1, 20.0, 3.5)

    # Substract the correct parameters from the dataset for each subcatchment
    name = file_name
    area = data_set[file_name]['area_m2'].values[0]
    precipitation = data_set[file_name]['precipitation'].values
    evaporation =  data_set[file_name]['average_evap'].values

    root_finder = PegasusPython()  # Use the default parameters
    numerical_approximation = ImplicitEulerPython(root_finder)

    # GR4J implementation according the resource of SuperflexPy
    interception_filter = InterceptionFilter(id='ir')

    production_store = ProductionStore(parameters={'x1': x1, 'alpha': 2.0,
                                                   'beta': 5.0, 'ni': 4/9},
                                       states={'S0': 5.0},
                                       approximation=numerical_approximation,
                                       id='ps')

    splitter = Splitter(weight=[[0.9], [0.1]],
                        direction=[[0], [0]],
                        id='spl')

    unit_hydrograph_1 = UnitHydrograph1(parameters={'lag-time': x4},
                                        states={'lag': None},
                                        id='uh1')

    unit_hydrograph_2 = UnitHydrograph2(parameters={'lag-time': 2*x4},
                                        states={'lag': None},
                                        id='uh2')

    routing_store = RoutingStore(parameters={'x2': x2, 'x3': x3,
                                             'gamma': 5.0, 'omega': 3.5},
                                 states={'S0': 20.0},
                                 approximation=numerical_approximation,
                                 id='rs')

    transparent = Transparent(id='tr')

    junction = Junction(direction=[[0, None],  # First output
                                   [1, None],  # Second output
                                   [None, 0]], # Third output
                        id='jun')

    flux_aggregator = FluxAggregator(id='fa')

    catchment_element = Unit(layers=[[interception_filter],
                     [production_store],
                     [splitter],
                     [unit_hydrograph_1, unit_hydrograph_2],
                     [routing_store, transparent],
                     [junction],
                     [flux_aggregator]],
             id=f'unit-{name}')
    
    
    current_node = Node(
    units=[catchment_element],
    weights = [1.0],
    area=area,
    id=f'node-{name}'
    )
    
    current_node.set_input([evaporation, precipitation])
    
    return current_node

In [5]:
# Guesses for the parameters

x1, x2, x3, x4 = 50.0, 0.7, 15., 2.4

In [6]:
# Test nodes to see if the GR4J_element is working for the different subcatchments

node_1 = GR4J_element(x1, x2, x3, x4, datasets, '1021')
node_2 = GR4J_element(x1, x2, x3, x4, datasets, '632')
node_3 = GR4J_element(x1, x2, x3, x4, datasets, '144')


In [7]:
# Test network from test nodes

net_test = Network(
    nodes = [node_1, node_2, node_3],
    topology= {
    'node-1021':'node-632',
    'node-144':'node-632',
    'node-632': None 
    }
)

In [8]:
# Output of test network

net_test.reset_states()
net_test.set_timestep(1.0)
print(net_test._content_pointer)
output_test = net_test.get_output()
print(output_test)

{'node-1021': 0, 'node-632': 1, 'node-144': 2}
{'node-1021': [array([4.03861456, 1.82052126, 1.05528861, ..., 0.16828827, 0.15597032,
       0.1437216 ])], 'node-144': [array([4.03861456, 1.82052126, 1.05515694, ..., 0.15659111, 0.14571798,
       0.13478985])], 'node-632': [array([4.03861456, 1.82052126, 1.05492933, ..., 0.15693422, 0.1461062 ,
       0.13518954])]}


In [9]:
# add all the GR4J elements in one dictionary with the corresponding subcatchment name

nodes_dict = {}

for filename in file_identifier_list:
    # Call GR4J_element function for the current filename
    node = GR4J_element(x1, x2, x3, x4, datasets, filename)
    # Add the Node object to the dictionary with filename as key
    nodes_dict[filename] = node


In [10]:
# Open the network file of the cathcments 

connection_file = glob.glob(os.path.join(data_path, '*.yaml'))

with open(connection_file[0], 'r') as yaml_file:
    full_network = yaml.load(yaml_file, Loader=yaml.FullLoader)

In [11]:
# Rename the names of the network file, so it can be used for GR4J

new_full_network = {}
keys_set = set()

for key, value in full_network.items():
    new_key = key.split('_')[1].split('.')[0]  # Extract the number from the key
    new_value = value.split('_')[1].split('.')[0]  # Extract the number from the value
    
    # Check for duplicate keys in the file
    if new_key in keys_set:
        print(f"Duplicate key found: {new_key}")
        # Handle duplicate key here, such as skipping or updating existing entry
    else:
        keys_set.add(new_key)
        new_full_network[f'node-{new_key}'] = f'node-{new_value}'

# Convert the set of keys into a list
keys_list = list(keys_set)

new_full_network

{'node-1021': 'node-1048',
 'node-1027': 'node-851',
 'node-1048': 'node-955',
 'node-1076': 'node-1112',
 'node-1086': 'node-1048',
 'node-1091': 'node-1091',
 'node-1096': 'node-908',
 'node-1101': 'node-1101',
 'node-1107': 'node-1101',
 'node-1112': 'node-1101',
 'node-1113': 'node-1076',
 'node-1114': 'node-1096',
 'node-1130': 'node-1344',
 'node-1131': 'node-1251',
 'node-1156': 'node-1101',
 'node-1169': 'node-1169',
 'node-1173': 'node-1378',
 'node-12': 'node-206',
 'node-1207': 'node-1769',
 'node-1209': 'node-1769',
 'node-1220': 'node-1114',
 'node-1223': 'node-1220',
 'node-1236': 'node-1027',
 'node-1249': 'node-1173',
 'node-1251': 'node-1251',
 'node-1259': 'node-1251',
 'node-1266': 'node-1344',
 'node-1272': 'node-1378',
 'node-1286': 'node-1353',
 'node-13': 'node-1424',
 'node-1344': 'node-1130',
 'node-1353': 'node-1101',
 'node-1378': 'node-1378',
 'node-1424': 'node-1173',
 'node-144': 'node-267',
 'node-1520': 'node-1378',
 'node-161': 'node-325',
 'node-1665':

In [12]:
# Replace the nodes that end to the same node for a None value to complete the Network

adjusted_network = {}

for key, value in new_full_network.items():
    if key == value:
        adjusted_network[key] = None
    else:
        adjusted_network[key] = value

print(adjusted_network)

# Count the number of None values in the Network to determine the number of outflows
count_none = sum(1 for value in adjusted_network.values() if value is None)
# print(count_none)

{'node-1021': 'node-1048', 'node-1027': 'node-851', 'node-1048': 'node-955', 'node-1076': 'node-1112', 'node-1086': 'node-1048', 'node-1091': None, 'node-1096': 'node-908', 'node-1101': None, 'node-1107': 'node-1101', 'node-1112': 'node-1101', 'node-1113': 'node-1076', 'node-1114': 'node-1096', 'node-1130': 'node-1344', 'node-1131': 'node-1251', 'node-1156': 'node-1101', 'node-1169': None, 'node-1173': 'node-1378', 'node-12': 'node-206', 'node-1207': 'node-1769', 'node-1209': 'node-1769', 'node-1220': 'node-1114', 'node-1223': 'node-1220', 'node-1236': 'node-1027', 'node-1249': 'node-1173', 'node-1251': None, 'node-1259': 'node-1251', 'node-1266': 'node-1344', 'node-1272': 'node-1378', 'node-1286': 'node-1353', 'node-13': 'node-1424', 'node-1344': 'node-1130', 'node-1353': 'node-1101', 'node-1378': None, 'node-1424': 'node-1173', 'node-144': 'node-267', 'node-1520': 'node-1378', 'node-161': 'node-325', 'node-1665': 'node-1353', 'node-1745': 'node-1378', 'node-1769': 'node-1745', 'node-

In [13]:
# Create a list for all nodes in the network

nodes_list_gr4j = []

for number in keys_list:
    filename = f'node_{number}.gpkg'
    node = GR4J_element(x1, x2, x3, x4, datasets, number)
    nodes_list_gr4j.append(node)
nodes_list_gr4j[0]

Module: superflexPy
Node: node-1665
Units:
	['unit-1665']
Weights:
	[1.0]
********************
********************
Module: superflexPy
Unit: unit-1665
Layers:
	[['ir'], ['ps'], ['spl'], ['uh1', 'uh2'], ['rs', 'tr'], ['jun'], ['fa']]
********************
Module: superflexPy
Element: ir

********************
Module: superflexPy
Element: ps
Parameters:
	unit-1665_ps_x1 : 50.0
	unit-1665_ps_alpha : 2.0
	unit-1665_ps_beta : 5.0
	unit-1665_ps_ni : 0.4444444444444444
States:
	node-1665_unit-1665_ps_S0 : 5.0

********************
Module: superflexPy
Element: spl
Weight:
	[[0.9], [0.1]]
Direction:
	[[0], [0]]

********************
Module: superflexPy
Element: uh1
Parameters:
	unit-1665_uh1_lag-time : 2.4
States:
	node-1665_unit-1665_uh1_lag : None

********************
Module: superflexPy
Element: uh2
Parameters:
	unit-1665_uh2_lag-time : 4.8
States:
	node-1665_unit-1665_uh2_lag : None

********************
Module: superflexPy
Element: rs
Parameters:
	unit-1665_rs_x2 : 0.7
	unit-1665_rs_x3 : 1

In [14]:
# try to select a part from the network to check for numerical stability

selected_section = {key: value for i, (key, value) in enumerate(adjusted_network.items()) if i < 4}
print(selected_section)


{'node-1021': 'node-1048', 'node-1027': 'node-851', 'node-1048': 'node-955', 'node-1076': 'node-1112'}


In [15]:
# The network for GR4J

net = Network(
    nodes = nodes_list_gr4j,
    topology= adjusted_network)

In [16]:
net.reset_states()
net.set_timestep(1.0)
print(net._content_pointer)
output = net.get_output()
print(output)

{'node-1665': 0, 'node-315': 1, 'node-908': 2, 'node-269': 3, 'node-668': 4, 'node-436': 5, 'node-333': 6, 'node-599': 7, 'node-1909': 8, 'node-875': 9, 'node-341': 10, 'node-439': 11, 'node-296': 12, 'node-443': 13, 'node-372': 14, 'node-622': 15, 'node-373': 16, 'node-619': 17, 'node-220': 18, 'node-746': 19, 'node-946': 20, 'node-885': 21, 'node-1272': 22, 'node-251': 23, 'node-379': 24, 'node-958': 25, 'node-1107': 26, 'node-640': 27, 'node-864': 28, 'node-620': 29, 'node-777': 30, 'node-617': 31, 'node-420': 32, 'node-421': 33, 'node-1207': 34, 'node-788': 35, 'node-440': 36, 'node-434': 37, 'node-1209': 38, 'node-871': 39, 'node-458': 40, 'node-948': 41, 'node-313': 42, 'node-416': 43, 'node-614': 44, 'node-872': 45, 'node-703': 46, 'node-409': 47, 'node-319': 48, 'node-689': 49, 'node-272': 50, 'node-1220': 51, 'node-144': 52, 'node-649': 53, 'node-1113': 54, 'node-1259': 55, 'node-635': 56, 'node-398': 57, 'node-838': 58, 'node-945': 59, 'node-267': 60, 'node-357': 61, 'node-35

RuntimeError: module : superflexPy, solver : PegasusPython, Error message : not converged. iter_max : 10000